# Use Snowpark_OpenAI Notebook template

## Import Snowpark Pandas for dumping csv to Snowflake table

In [ ]:
! pip install "snowflake-connector-python[pandas]"
! sudo pip install snowflake-ml-python==1.0.11 -U
! pip install --upgrade snowflake-snowpark-python==1.9.0
! pip install --upgrade xgboost==1.7.3
! pip install --upgrade numpy==1.24.3
! pip install --upgrade pandas==1.5.3
! pip install --upgrade anyio==3.5.0
! pip install --upgrade packaging==23.1
! pip install --upgrade scikit-learn==1.3.0
! pip install --upgrade typing-extensions==4.7.1
! pip install --upgrade cryptography==39.0.0
! pip install --upgrade fsspec==2023.9.2
!pip install refractml

  Using cached https://files.pythonhosted.org/packages/7a/36/d01948e9b486e40e4988db00455bf30718139d9dc98d10536f1e68385039/snowflake_connector_python-3.9.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e5/3e/741d8c82801c347547f8a2a06aa57dbb1992be9e948df2ea0eda2c8b79e8/idna-3.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/fa/057f9d7a5364c86ccb6a4bd4e5c58920dcb66532be0cc21da3f9c7617ec3/cryptography-42.0.5-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/54/a7/2104f674a5a6845b04c8ff01659becc6b8978ca410b82b94287e0b1e018b/pyOpenSSL-24.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2b/4f/e04a8067c7c96c364cef7ef73906504e2f40d690811c021e1a1901473a19

In [1]:
from snowflake.snowpark.session import Session
import pandas as pd
import subprocess

## Code to establish connection and read and dump csv as a snowflake table (handling null values before loading)

In [2]:
#Import all snowflake connection details from template variables.

db_user = 'SPECTRA_DEMO_USER'
db_password = 'Spectra@2023'
db_account = 'fya62509.us-east-1'
db_database =  'SPECTRA_DEMO'
db_role = 'RL_TEST_FOSFOR_SPECTRA'
db_warehouse = 'WH_TEST_FOSFOR_SPECTRA'
db_schema = 'AUTO_INSURANCE_CLAIMS_DW'

In [3]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session = Session.builder.configs(connection_params).create()

In [4]:
import pandas as pd
session.sql('use warehouse WH_TEST_FOSFOR_SPECTRA;').collect()
session.sql('use database SPECTRA_DEMO;').collect()
session.sql('use schema SPECTRA_DEMO.AUTO_INSURANCE_CLAIMS_DW;').collect()

STG_CUSTOMER = session.table('SPECTRA_DEMO.AUTO_INSURANCE_CLAIMS_DW.STG_CUSTOMER').to_pandas()
STG_CLAIM = session.table('SPECTRA_DEMO.AUTO_INSURANCE_CLAIMS_DW.STG_CLAIM').to_pandas()
STG_INCIDENT = session.table('SPECTRA_DEMO.AUTO_INSURANCE_CLAIMS_DW.STG_INCIDENT').to_pandas()
STG_HARMONIZE_AI_CLAIMS = session.table('SPECTRA_DEMO.AUTO_INSURANCE_CLAIMS_DW.STG_HARMONIZE_AI_CLAIMS').to_pandas()
STG_POLICY = session.table('SPECTRA_DEMO.AUTO_INSURANCE_CLAIMS_DW.STG_POLICY').to_pandas()

In [5]:
STG_CUSTOMER.to_csv('/data/ILF_STG_CUSTOMER.csv')
STG_CLAIM.to_csv('/data/ILF_STG_CLAIM.csv')
STG_INCIDENT.to_csv('/data/ILF_STG_INCIDENT.csv')
STG_HARMONIZE_AI_CLAIMS.to_csv('/data/ILF_STG_HARMONIZE_AI_CLAIMS.csv')
STG_POLICY.to_csv('/data/ILF_STG_POLICY.csv')

# Code to establish connection and  dump csv as a snowflake table in FDC setup

In [56]:
#Import all snowflake connection details from template variables.
db_user = 'MANISH'
db_password = 'Password@2023'
db_account = 'ug94937.us-east4.gcp'
db_database =  'FDC_DATA_MANISH'
db_role = 'MANISH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'PUBLIC'

In [57]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

In [58]:
session1.sql('use warehouse FOSFOR_SOLUTIONS_WH;').collect()
session1.sql('use database FDC_DATA_MANISH;').collect()
session1.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

[Row(status='Statement executed successfully.')]

In [59]:
DF_STG_CUSTOMER=session1.createDataFrame(
        STG_CUSTOMER.values.tolist(),
        schema=STG_CUSTOMER.columns.tolist())
DF_STG_CUSTOMER.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.ILF_STG_CUSTOMER")

In [61]:
DF_STG_CLAIM=session1.createDataFrame(
        STG_CLAIM.values.tolist(),
        schema=STG_CLAIM.columns.tolist())
DF_STG_CLAIM.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.ILF_STG_CLAIM")

In [62]:
DF_STG_INCIDENT=session1.createDataFrame(
        STG_INCIDENT.values.tolist(),
        schema=STG_INCIDENT.columns.tolist())
DF_STG_INCIDENT.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.ILF_STG_INCIDENT")

In [63]:
DF_STG_HARMONIZE_AI_CLAIMS=session1.createDataFrame(
        STG_HARMONIZE_AI_CLAIMS.values.tolist(),
        schema=STG_HARMONIZE_AI_CLAIMS.columns.tolist())
DF_STG_HARMONIZE_AI_CLAIMS.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.ILF_STG_HARMONIZE_AI_CLAIMS")

In [64]:
DF_STG_POLICY=session1.createDataFrame(
        STG_POLICY.values.tolist(),
        schema=STG_POLICY.columns.tolist())
DF_STG_POLICY.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.STG_POLICY")

In [65]:
DF_STG_CLAIM=session1.createDataFrame(
        STG_CLAIM.values.tolist(),
        schema=STG_CLAIM.columns.tolist())
DF_STG_CLAIM.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.STG_CLAIM")